In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import random
import time
import re
import os

In [14]:
categories = ['AI/ML', 'Python', 'Data+Engineer', 'Data+Science'] 
all_links = set() 
vacancies = []

In [15]:
def scrape_dou_jobs(category):
    chrome_options = Options()
    # chrome_options.add_argument("--headless")

    chrome_options.add_argument("--window-size=900,1000")
    chrome_options.add_argument("--window-position=960,0")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    url = f"https://jobs.dou.ua/vacancies/?category={category}"
    driver.get(url)

    try:
        while True:
            try:
                more_button = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".more-btn a"))
                )
                more_button.click()
                time.sleep(2)
            except:
                break

        vacancies_elements = driver.find_elements(By.CSS_SELECTOR, "li.l-vacancy")
        
        page_results = []
        for v in vacancies_elements:
            try:
                title_el = v.find_element(By.CSS_SELECTOR, "a.vt")
                company_el = v.find_element(By.CSS_SELECTOR, "a.company")
                
                link = title_el.get_attribute("href")
                
                item = {
                    "title": title_el.text,
                    "company": company_el.text,
                    "link": link,
                    "source_category": category
                }
                page_results.append(item)
            except:
                continue
                
        return page_results

    finally:
        driver.quit()

In [16]:
for cat in categories:
    raw_results = scrape_dou_jobs(cat)
    
    added_count = 0
    skipped_count = 0
    
    for job in raw_results:
        url = job['link']
        
        if url not in all_links:
            all_links.add(url)
            vacancies.append(job)
            added_count += 1
        else:
            skipped_count += 1
            
    print(f"Знайдено: {len(raw_results)}. Додано нових: {added_count}. Дублікатів: {skipped_count}.\n")

Знайдено: 242. Додано нових: 242. Дублікатів: 0.

Знайдено: 226. Додано нових: 204. Дублікатів: 22.

Знайдено: 97. Додано нових: 90. Дублікатів: 7.

Знайдено: 72. Додано нових: 47. Дублікатів: 25.



In [17]:
def show_stats(vacs_list, count):
    safe_count = min(count, len(vacs_list))
    
    for i, v in enumerate(vacs_list[:safe_count]):
        company_name = v['company'].strip()
        print(f"{i+1}. {v['title']} '{company_name}'")
        print(f"   Category: {v.get('source_category', '-')}")

In [18]:
show_stats(vacancies, count=10)

1. RnD Engineer (CV + ML) 'Warbirds'
   Category: AI/ML
2. AI Video Creator 'HOLYWATER TECH'
   Category: AI/ML
3. Motion Designer / AI Video Editor — PawChamp 'SKELAR'
   Category: AI/ML
4. Lead AI Engineer 'Spendbase'
   Category: AI/ML
5. Senior AI Engineer 'SIXT'
   Category: AI/ML
6. AI Video Creator 'AMO'
   Category: AI/ML
7. Senior Deep Learning Engineer (Computer Vision) 'Ajax Systems'
   Category: AI/ML
8. AI Designer Lead 'FREITTY'
   Category: AI/ML
9. AI Researcher 'Fuelfinance'
   Category: AI/ML
10. AI Lead 'OBRIO'
   Category: AI/ML


In [19]:
df = pd.DataFrame(vacancies)

total = len(df)

df['title'] = df['title'].str.strip()
df['char_length'] = df['title'].apply(len)
df['word_length'] = df['title'].apply(lambda x: len(x.split()))

mean_char = df['char_length'].mean()
med_char = df['char_length'].median()
mean_word = df['word_length'].mean()
med_word = df['word_length'].median()
counts = df['source_category'].value_counts()
percents = df['source_category'].value_counts(normalize=True)

print(f"Вакансій: {total}")

for cat in counts.index:
    count = counts[cat]
    pct = percents[cat] * 100
    print(f"{cat:<15} {count:>3} ({pct:>4.1f}%)")

print(f"\nСимволи: {mean_char:>6.1f} | {med_char:<4.1f}")
print(f"Слова:   {mean_word:>6.1f} | {med_word:<4.1f}")

Вакансій: 583
AI/ML           242 (41.5%)
Python          204 (35.0%)
Data+Engineer    90 (15.4%)
Data+Science     47 ( 8.1%)

Символи:   32.1 | 30.0
Слова:      4.4 | 4.0 


In [20]:
def clean_text(text):
    if not isinstance(text, str):
        return str(text)
    
    text = text.replace("’", "'").replace("`", "'").replace("‘", "'")
    text = re.sub(r'https?://\S+|www\.\S+', '<URL>', text)
    text = re.sub(r'\S+@\S+\.\S+', '<EMAIL>', text)
    text = re.sub(r'\+?[\d\-\(\)\s]{9,}', ' <PHONE> ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

test_str = "Send CV to  hr@company.com or visit https://dou.ua . Call +380 50 123 45 67.  We’re hiring! "
print(f"ДО:    '{test_str}'")
print(f"ПІСЛЯ: '{clean_text(test_str)}'")

df['title_cleaned'] = df['title'].apply(clean_text)

print(df[['title', 'title_cleaned']].head(5))

ДО:    'Send CV to  hr@company.com or visit https://dou.ua . Call +380 50 123 45 67.  We’re hiring! '
ПІСЛЯ: 'Send CV to <EMAIL> or visit <URL> . Call <PHONE> . We're hiring!'
                                          title  \
0                        RnD Engineer (CV + ML)   
1                              AI Video Creator   
2  Motion Designer / AI Video Editor — PawChamp   
3                              Lead AI Engineer   
4                            Senior AI Engineer   

                                  title_cleaned  
0                        RnD Engineer (CV + ML)  
1                              AI Video Creator  
2  Motion Designer / AI Video Editor — PawChamp  
3                              Lead AI Engineer  
4                            Senior AI Engineer  


In [21]:
def get_description(driver, url):
    try:
        driver.get(url)
        time.sleep(random.uniform(1.0, 2.0)) 
        desc_element = driver.find_element(By.CSS_SELECTOR, ".b-vacancy")
        return desc_element.text
    except Exception as e:
        return ""

In [22]:
chrome_options = Options()
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=900,1000")
chrome_options.add_argument("--window-position=960,0")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

try:
    for i, job in enumerate(vacancies):
        if 'description' in job and len(job['description']) > 10:
            continue
            
        url = job['link']
        desc_text = get_description(driver, url)
        job['description'] = desc_text

        if (i + 1) % 10 == 0:
            print(f"[{i+1}/{len(vacancies)}] Опрацьовано...")

except KeyboardInterrupt:
    print("\nСкрипт зупинено користувачем. Дані, що встигли зібрати - збережені.")

finally:
    driver.quit()

[10/583] Опрацьовано...
[20/583] Опрацьовано...
[30/583] Опрацьовано...
[40/583] Опрацьовано...
[50/583] Опрацьовано...
[60/583] Опрацьовано...
[70/583] Опрацьовано...
[80/583] Опрацьовано...
[90/583] Опрацьовано...
[100/583] Опрацьовано...
[110/583] Опрацьовано...
[120/583] Опрацьовано...
[130/583] Опрацьовано...
[140/583] Опрацьовано...
[150/583] Опрацьовано...
[160/583] Опрацьовано...
[170/583] Опрацьовано...
[180/583] Опрацьовано...
[190/583] Опрацьовано...
[200/583] Опрацьовано...
[210/583] Опрацьовано...
[220/583] Опрацьовано...
[230/583] Опрацьовано...
[240/583] Опрацьовано...
[250/583] Опрацьовано...
[260/583] Опрацьовано...
[270/583] Опрацьовано...
[280/583] Опрацьовано...
[290/583] Опрацьовано...
[300/583] Опрацьовано...
[310/583] Опрацьовано...
[320/583] Опрацьовано...
[330/583] Опрацьовано...
[340/583] Опрацьовано...
[350/583] Опрацьовано...
[360/583] Опрацьовано...
[370/583] Опрацьовано...
[380/583] Опрацьовано...
[390/583] Опрацьовано...
[400/583] Опрацьовано...
[410/583]

In [26]:
os.makedirs('../data', exist_ok=True)

df_raw = pd.DataFrame(vacancies)

raw_path = '../data/raw.csv'
df_raw.to_csv(raw_path, index=False)

print(f"Сирі дані збережено у '{raw_path}'")
print(f"Розмір: {df_raw.shape}")
print(df_raw.head(3))

Сирі дані збережено у '../data/raw.csv'
Розмір: (583, 5)
                                          title          company  \
0                        RnD Engineer (CV + ML)         Warbirds   
1                              AI Video Creator   HOLYWATER TECH   
2  Motion Designer / AI Video Editor — PawChamp           SKELAR   

                                                link source_category  \
0  https://jobs.dou.ua/companies/warbirds-boiovi-...           AI/ML   
1  https://jobs.dou.ua/companies/holy-water/vacan...           AI/ML   
2  https://jobs.dou.ua/companies/skelar/vacancies...           AI/ML   

                                         description  
0  Warbirds Бойові Птахи України Всі вакансії ком...  
1  HOLYWATER TECH Всі вакансії компанії\nHOLYWATE...  
2  SKELAR Всі вакансії компанії\nМи venture build...  


In [27]:
df_raw['desc_char_len'] = df_raw['description'].astype(str).apply(len)
df_raw['desc_word_len'] = df_raw['description'].astype(str).apply(lambda x: len(x.split()))


print(f"Mean:   {df_raw['desc_word_len'].mean():.1f}")
print(f"Median:  {df_raw['desc_word_len'].median():.1f}")

duplicates = df_raw.duplicated(subset=['description']).sum()
trash_count = len(df_raw[df_raw['desc_word_len'] < 10])

print(f"Точні дублікати описів: {duplicates} ({(duplicates/len(df_raw))*100:.1f}%)")
print(f"Занадто короткі (<10 слів): {trash_count} ({(trash_count/len(df_raw))*100:.1f}%)")

Mean:   463.3
Median:  434.0
Точні дублікати описів: 0 (0.0%)
Занадто короткі (<10 слів): 0 (0.0%)


In [ ]:
df_raw['description_cleaned'] = df_raw['description'].astype(str).apply(clean_text)

# for future
df_processed = df_raw[df_raw['desc_word_len'] >= 10].copy()

final_cols = ['link', 'title', 'source_category', 'description_cleaned']
df_processed = df_processed[final_cols]

processed_path = '../data/processed.csv'
df_processed.to_csv(processed_path, index=False)

print(f"Очищені дані збережено у '{processed_path}'")
print(f"Кількість вакансій після очистки: {len(df_processed)}")
print("\nПриклад очищеного тексту:")
print(df_processed['description_cleaned'].iloc[0][:300] + "...")

Очищені дані збережено у '../data/processed.csv'
Кількість вакансій після очистки: 583

Приклад очищеного тексту:
Warbirds Бойові Птахи України Всі вакансії компанії Компанія «Бойові Птахи України» (Warbirds.com.ua) є одним з провідних вітчизняних розробників та виробників БПЛА для потреб Сил Оборони України. Головний продукт компанії — безпілотний авіаційний комплекс «ВАЛК-1», відомий як «Валькірія», на фронт ...


### Висновок

**Характеристика даних:**
Було зібрано та проаналізовано **583 вакансії** з сайту DOU за категоріями AI/ML, Python, Data Engineer та Data Science. Дані очищено від HTML-тегів, а середня довжина опису складає близько **460 слів**, що є достатнім обсягом для виділення сутностей (NER).

**Якість даних:**
Якість зібраного датасету висока: виявлено **0% точних дублікатів** та **0% "сміттєвих"** (занадто коротких) вакансій. Це свідчить про коректну роботу скрапера на базі Selenium, який успішно відфільтрував пусті сторінки.

**Ризики та плани:**
Основний ризик для майбутньої моделі - це **змішана мова описів** (частина вакансій англійською, частина українською, або мікс в одному тексті). Також у текстах трапляються злиті списки, які потребуватимуть кращого розбиття на речення.
**Що доробити:** У наступній лабораторній роботі (Lab 2) планується вдосконалити токенізацію та розробити правила для коректного розділення речень у змішаних текстах.

In [2]:
df = pd.read_csv('../data/processed.csv')

df['raw_text'] = df['description_cleaned']

df['processed_text'] = df['raw_text'].str.lower()

df.to_csv('../data/lab2_dataset.csv', index=False)

display(df[['raw_text', 'processed_text']].head(3))

,raw_text,processed_text
0,Warbirds Бойові Птахи України Всі вакансії ком...,warbirds бойові птахи україни всі вакансії ком...
1,HOLYWATER TECH Всі вакансії компанії HOLYWATER...,holywater tech всі вакансії компанії holywater...
2,SKELAR Всі вакансії компанії Ми venture builde...,skelar всі вакансії компанії ми venture builde...
